In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import pandas as pd
from src.paths import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR / 'tabular_data.parquet')
df

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_next_27_hour,rides_next_28_hour,rides_next_29_hour,rides_next_30_hour,rides_next_31_hour,rides_next_32_hour,rides_next_33_hour,rides_next_34_hour,rides_next_35_hour,rides_next_36_hour
0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,10.0
1,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,4.0,0.0,0.0,0.0,1.0,2.0,5.0,2.0,2.0,2.0
2,0.0,1.0,2.0,0.0,0.0,2.0,1.0,1.0,1.0,2.0,...,0.0,0.0,0.0,0.0,2.0,0.0,5.0,4.0,1.0,8.0
3,4.0,2.0,2.0,1.0,0.0,0.0,0.0,2.0,1.0,4.0,...,2.0,1.0,1.0,0.0,1.0,1.0,7.0,2.0,2.0,3.0
4,0.0,0.0,1.0,0.0,0.0,0.0,3.0,1.0,3.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111211,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0
111212,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,2.0,3.0,0.0,3.0,0.0
111213,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,1.0,1.0
111214,0.0,0.0,1.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,2.0,0.0,0.0


In [4]:
from datetime import datetime
from src.data_split import train_test_split

X_train, y_train, X_test, y_test = train_test_split(
    df,
    cutoff_date=datetime(2022, 6, 1, 0, 0, 0),
    targets_columns_names=[c for c in df.columns if c.startswith('rides_next_')]
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

X_train.shape=(40713, 674)
y_train.shape=(40713, 36)
X_test.shape=(70503, 674)
y_test.shape=(70503, 36)


In [5]:
import numpy as np
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error
import optuna

from src.model import get_pipeline

def objective(trial: optuna.trial.Trial) -> float:
    """
    Given a set of hyper-parameters, it trains a model and computes an average
    validation error based on a TimeSeriesSplit
    """
    # pick hyper-parameters
    hyperparams = {
        "metric": 'mae',
        "verbose": -1,
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 3, 100),   
    }
       
    tss = TimeSeriesSplit(n_splits=2)
    scores = []
    for train_index, val_index in tss.split(X_train):

        # split data for training and validation
        X_train_, X_val_ = X_train.iloc[train_index, :], X_train.iloc[val_index,:]
        y_train_, y_val_ = y_train.iloc[train_index], y_train.iloc[val_index]
        
        # train the model
        pipeline = get_pipeline(**hyperparams)
        pipeline.fit(X_train_, y_train_)
        
        # evaluate the model
        y_pred = pipeline.predict(X_val_)
        mae = mean_absolute_error(y_val_, y_pred)

        scores.append(mae)
   
    # Return the mean score
    return np.array(scores).mean()

In [6]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=5)

[I 2023-09-03 20:03:34,893] A new study created in memory with name: no-name-de8f95d6-914c-40e8-8363-a149c147c401
[I 2023-09-03 20:06:07,522] Trial 0 finished with value: 0.5357410238570299 and parameters: {'num_leaves': 110, 'feature_fraction': 0.3159644364620229, 'bagging_fraction': 0.6430982887189218, 'min_child_samples': 59}. Best is trial 0 with value: 0.5357410238570299.
[I 2023-09-03 20:09:00,277] Trial 1 finished with value: 0.5355443754329897 and parameters: {'num_leaves': 199, 'feature_fraction': 0.2676844181074463, 'bagging_fraction': 0.8483907851183174, 'min_child_samples': 24}. Best is trial 1 with value: 0.5355443754329897.
[I 2023-09-03 20:13:08,872] Trial 2 finished with value: 0.5356831342072661 and parameters: {'num_leaves': 194, 'feature_fraction': 0.7221591738533804, 'bagging_fraction': 0.20992159592618195, 'min_child_samples': 11}. Best is trial 1 with value: 0.5355443754329897.
[I 2023-09-03 20:15:21,833] Trial 3 finished with value: 0.535659216021227 and paramete

In [7]:
best_params = study.best_trial.params
print(f'{best_params=}')

best_params={'num_leaves': 199, 'feature_fraction': 0.2676844181074463, 'bagging_fraction': 0.8483907851183174, 'min_child_samples': 24}


In [8]:
pipeline = get_pipeline(**best_params)
pipeline.fit(X_train, y_train)

[LightGBM] [Warning] feature_fraction is set=0.2676844181074463, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2676844181074463
[LightGBM] [Warning] bagging_fraction is set=0.8483907851183174, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8483907851183174
[LightGBM] [Warning] feature_fraction is set=0.2676844181074463, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2676844181074463
[LightGBM] [Warning] bagging_fraction is set=0.8483907851183174, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8483907851183174
[LightGBM] [Info] Total Bins 12130
[LightGBM] [Info] Number of data points in the train set: 40713, number of used features: 677
[LightGBM] [Info] Start training from score 0.416796
[LightGBM] [Warning] feature_fraction is set=0.2676844181074463, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2676844181074463
[LightGBM] [Warning] bagging_fraction is set=0.848390785118317

Pipeline(steps=[('functiontransformer-1',
                 FunctionTransformer(func=<function average_rides_last_4_weeks at 0x000001D52D424CA0>)),
                ('functiontransformer-2',
                 FunctionTransformer(func=<function latitude_and_longitude at 0x000001D52E0E4670>)),
                ('temporalfeaturesengineer', TemporalFeaturesEngineer()),
                ('multioutputregressor',
                 MultiOutputRegressor(estimator=LGBMRegressor(bagging_fraction=0.8483907851183174,
                                                              feature_fraction=0.2676844181074463,
                                                              force_col_wise=True,
                                                              min_child_samples=24,
                                                              num_leaves=199)))])

In [9]:
predictions = pipeline.predict(X_test)
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')

[LightGBM] [Warning] feature_fraction is set=0.2676844181074463, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2676844181074463
[LightGBM] [Warning] bagging_fraction is set=0.8483907851183174, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8483907851183174
[LightGBM] [Warning] feature_fraction is set=0.2676844181074463, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2676844181074463
[LightGBM] [Warning] bagging_fraction is set=0.8483907851183174, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8483907851183174
[LightGBM] [Warning] feature_fraction is set=0.2676844181074463, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2676844181074463
[LightGBM] [Warning] bagging_fraction is set=0.8483907851183174, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8483907851183174
[LightGBM] [Warning] feature_fraction is set=0.2676844181074463, colsample_bytree=1.0 will be ign

In [10]:
# Como da algun valor negativo como resultado eliminamos las predicciones negativas y los intervalos negativos reemplazando por cero
import numpy as np

#Limitar los valores predichos a un mínimo de cero
predictions = np.clip(predictions, 0, None)

In [11]:
from sklearn.metrics import mean_absolute_error
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')

test_mae=0.6811


In [15]:
from src.plot import plot_one_sample

plot_one_sample(
    features=X_test,
    targets=y_test,
    example_id=500,
    predictions=pd.DataFrame(predictions)
)